<a href="https://colab.research.google.com/github/Monakairine/Astromatch/blob/astromatch/Devoro_mona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
%load_ext google.colab.data_table

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


In [33]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from chart_studio import plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

%matplotlib inline
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

# Set seeds to make the experiment more reproducible.
import tensorflow as tf
from numpy.random import seed
tf.random.set_seed(1)

# Nova seção

In [ ]:
# Display query output immediately

%%bigquery --project devoro df
SELECT 
  * 
FROM `devoro.sales.ecommerce_sales_clean` limit 10000

In [ ]:
df.describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,id,marketplace_sale_id,date,brand,store_id,store_name,products,origin,sub_total_value,freight_value,...,is_pickup,address,client_name,client_phone,client_document_type,client_document,client_marketplace_id,client_birthdate,client_email,total_payment_value
count,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,...,10000,10000,10000,8853,10000,9354,7644,0,5100,10000
unique,10000,9996,9999,10,67,75,10000,5,918,73,...,2,7137,6904,6964,3,6588,7451,0,29,1736
top,1012803,-central,2022-06-11 20:46:44+00:00,Elettrica,424,MOEMA - ROUXINOL (FULL),"[{'id': '21624004', 'sku': '38411', 'name': 'B...",iFood,59,0,...,False,"{'country': 'Brasil', 'state': 'SP', 'city': '...",RAPPI,,CPF,26900161000125,8335809,NaN,,59
freq,1,4,2,3138,541,541,1,6525,216,7246,...,9962,1683,2210,1387,6829,2109,4,NaN,2417,141
first,NaN,NaN,2020-06-27 20:43:48+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,2022-08-16 21:30:38+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

duplicateRows = df[df.duplicated(['marketplace_sale_id'])]
duplicateRows

,id,marketplace_sale_id,date,brand,store_id,store_name,products,origin,sub_total_value,freight_value,...,is_pickup,address,client_name,client_phone,client_document_type,client_document,client_marketplace_id,client_birthdate,client_email,total_payment_value
1484,378013,-central,2020-11-16 17:07:29.740000+00:00,Lanchonete da Cidade,457,LC6 - PINHEIROS (FULL),"[{'id': '20386270', 'sku': '30825', 'name': 'X...",CENTRAL,64,0,...,False,"{'country': 'BR', 'state': 'SP', 'city': 'São ...",Eduardo Tadeu,+055 (11) 96435-1120,OTHER,,None,None,,64
3082,684826,-Central,2021-02-01 11:14:28.770000+00:00,Piraja,471,PRAINHA ITAIM (FULL),"[{'id': '20981171', 'sku': '30395', 'name': 'P...",CENTRAL,5,0,...,False,"{'country': 'BR', 'state': 'SP', 'city': 'São ...",Eduardo Tadeu,+055 (11) 99650-3530,OTHER,,None,None,eduardosilva@ciatc.com.br,5
5073,198706,-central,2020-09-18 16:24:11.650000+00:00,Braz Pizzaria,482,BRAZ PERDIZES (FULL),"[{'id': '20018997', 'sku': '39952', 'name': 'R...",CENTRAL,5,0,...,False,"{'country': 'BR', 'state': 'SP', 'city': 'São ...",LEANDRO UEDA VELLA,+055 (11) 99187-0447,OTHER,,None,None,,5
6739,191136,-central,2020-09-14 18:05:29.270000+00:00,Braz Pizzaria,494,BRAZ PARAISO (QUINTAL PARAISO) (FULL),"[{'id': '20003263', 'sku': '39952', 'name': 'R...",CENTRAL,5,0,...,False,"{'country': 'BR', 'state': 'SP', 'city': 'São ...",Ednaldo Santos (Teste),+055 (11) 96435-1043,OTHER,,None,None,,5


In [ ]:
df['store_id'].unique()

array(['477', '531', '752', '483', '467', '910', '970', '969', '920',
       '1176', '1219', '1177', '1175', '463', '520', '3259', '4302',
       '385', '393', '423', '424', '457', '458', '459', '464', '465',
       '466', '468', '469', '470', '471', '472', '473', '474', '475',
       '476', '481', '482', '484', '485', '486', '487', '489', '490',
       '491', '492', '494', '495', '496', '497', '499', '500', '505',
       '581', '608', '698', '738', '779', '878', '957', '965', '966',
       '967', '968', '971', '974', '1218'], dtype=object)

In [ ]:
df['origin'].unique() ## ifood e app são os melhores dados. App é app devoro

array(['Rappi', 'Uber Eats', 'iFood', 'APP', 'CENTRAL'], dtype=object)

In [ ]:
# da tabela order vem do back do aplicativo
#events vem do firebase -- tem só do app devoro
#user_id se tem um número user tava logado, se vem null não

In [ ]:
df['store_name'].unique() #FULL logistica não controlada pela devoro, MKT são 

array(['TATU - TATUAPE (FULL)', 'ASTOR OSCAR FREIRE (FULL)',
       'BRAZ TRATTORIA - PINHEIROS (FULL)', 'ORIGINAL (FULL) (PAI)',
       'PIRAJA ELDORADO (FULL)',
       'BRAZ TRATTORIA - PINHEIROS (IFOOD) (FULL)',
       'PIRAJA FARIA LIMA (MKT)', 'ASTOR OSCAR FREIRE (MKT)',
       'Devoro Empório', 'BRAZ TRATTORIA - PINHEIROS (MKT)',
       'BRAZ LAGOA (JB) (MKT)',
       'BRAZ TRATTORIA - PINHEIROS (IFOOD) (MKT)',
       'PIRAJA TAMBORE (FULL)', 'LC3 - MORUMBI (MKT)',
       'BRAZ VELOCE (BARRA RJ) (PAI VELOCE) (FULL)', 'ORIGINAL (FULL)',
       'PICO - PINHEIROS (MKT)',
       'BRAZ PINH/PERD (DEL LUMINARIAS) (MKT) (PAI)',
       'LC1 - JARDINS (FULL)', 'LC3 - MORUMBI (FULL)',
       'BRAZ MOEMA (DEL ROUXINOL) (MKT)', 'MOEMA - ROUXINOL (FULL)',
       'LC6 - PINHEIROS (FULL)', 'LC2 - MOEMA (FULL)',
       'LC4 - HIGIENÓPOLIS (FULL)', 'PICO - PINHEIROS (FULL)',
       'PIRAJA ALPHAVILLE (FULL)', 'PRAINHA PINHEIROS (FULL)',
       'PIRAJA FARIA LIMA (FULL)', 'PIRAJA MORUMBI (FULL)',
